In [1]:
!pip -q install streamlit

In [2]:
!pip -q install pyngrok

In [3]:
import cv2, os, random
from pyngrok import ngrok
from io import StringIO 
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from pyngrok import ngrok
import streamlit as st
import torch
import torchvision.transforms as T
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,concatenate,Input,LSTM, Reshape,Permute
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Conv1D,Conv2DTranspose,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,TensorBoard
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

In [4]:
%%writefile app.py
import cv2, os, random
from pyngrok import ngrok
from io import StringIO 
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from pyngrok import ngrok
import streamlit as st
import torch
import torchvision.transforms as T
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,concatenate,Input,LSTM, Reshape,Permute
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Conv1D,Conv2DTranspose,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,TensorBoard
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

# Unet Model Architecture
def unet(pretrained_weigths=None):
  input = Input(shape = (256,320,3))
  
  conv_1 = Conv2D(64,(3,3),activation='relu', padding = 'same')(input)
  conv_2 = Conv2D(64,(3,3),activation = 'relu', padding = 'same')(conv_1)
  pool_1 =  MaxPooling2D(pool_size =(2,2))(conv_2)
  pool_1 = Dropout(0.5)(pool_1)


  conv_3 = Conv2D(128,(3,3),activation = 'relu', padding = 'same')(pool_1)
  conv_4 = Conv2D(128,(3,3), activation ='relu', padding = 'same')(conv_3)
  pool_2 = MaxPooling2D(pool_size = (2,2))(conv_4)
  pool_2 = Dropout(0.5)(pool_2)


  conv_5 = Conv2D(256,(3,3),activation='relu', padding = 'same')(pool_2)
  conv_6 = Conv2D(256,(3,3),activation = 'relu', padding = 'same')(conv_5)
  pool_3 = MaxPooling2D(pool_size=(2,2))(conv_6)
  pool_3 = Dropout(0.5)(pool_3)

  conv_7 = Conv2D(512,(3,3),activation = 'relu', padding = 'same')(pool_3)
  conv_8 = Conv2D(512,(3,3),activation = 'relu', padding = 'same')(conv_7)
  pool_4 = MaxPooling2D(pool_size=(2,2))(conv_8)
  pool_4 = Dropout(0.5)(pool_4)


  conv_9 =Conv2D(1024,(3,3),activation = 'relu', padding = 'same')(pool_4)
  conv_10 = Conv2D(1024,(3,3),activation ='relu', padding = 'same')(conv_9)

  upconv_1 = Conv2DTranspose(512,(3,3),strides= (2,2),padding='same')(conv_10)
  concat_1 = concatenate([upconv_1,conv_8],axis=3)
  concat_1 = Dropout(0.5)(concat_1)
  
  conv_11 = Conv2D(512,(3,3),activation='relu', padding = 'same')(concat_1)
  conv_12 = Conv2D(512,(3,3),activation='relu', padding = 'same')(conv_11)

  upconv_2 = Conv2DTranspose(256,(3,3),strides = (2,2),padding='same')(conv_12)
  concat_2 = concatenate([upconv_2,conv_6],axis=3)
  concat_2 = Dropout(0.5)(concat_2)

  conv_13 = Conv2D(256,(3,3),activation = 'relu', padding = 'same')(concat_2)
  conv_14 = Conv2D(256,(3,3),activation = 'relu', padding = 'same')(conv_13)

  upconv_3 = Conv2DTranspose(128,(3,3),strides = (2,2),padding='same')(conv_14)
  concat_3 = concatenate([upconv_3,conv_4],axis=3)
  concat_3 = Dropout(0.5)(concat_3)

  conv_15 = Conv2D(128,(3,3),activation='relu', padding = 'same')(concat_3)
  conv_16 = Conv2D(128,(3,3),activation='relu', padding = 'same')(conv_15)

  upconv_4 = Conv2DTranspose(64,(3,3),strides = (2,2),padding='same')(conv_16)
  concat_4 = concatenate([upconv_4,conv_2],axis=3)
  concat_4 = Dropout(0.5)(concat_4)

  conv_17 = Conv2D(64,(3,3),activation='relu', padding = 'same')(concat_4)
  conv_18 =Conv2D(64,(3,3),activation='relu', padding = 'same')(conv_17)
  
  out = Conv2D(7,(1,1),padding='same',activation = 'softmax')(conv_18)
  model = Model(input,out)
  return model
model = unet()
model.load_weights('/content/drive/MyDrive/unet_best_model.h5')

def plot_predicted_mask(original_img,predicted_mask):
    """
    Inputs: image and mask 
    Outputs: plot both image and plot side by side
    """
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(7, 5))
    axes[0].imshow(original_img)
    axes[0].get_xaxis().set_visible(False)
    axes[0].get_yaxis().set_visible(False)
    axes[0].set_title('Original MRI')
    axes[1].imshow(predicted_mask)
    axes[1].get_xaxis().set_visible(False)
    axes[1].get_yaxis().set_visible(False)
    axes[1].set_title('Predicted Mask')
    fig.tight_layout()
    filename = 'pair' + str(random.randint(100,1000) ) + str(random.randint(100,1000) ) + '.png'
    plt.savefig(filename)
    print('File saved successully')
    print(filename)
    return  filename

def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

st.title('Indian Driving image segmentation')
st.markdown("***")

st.subheader('Upload the Image')
file_uploaded = st.file_uploader('Choose the file', type = ['jpg','png','jpeg'])
if file_uploaded is not None:
  orginal_image = Image.open(file_uploaded)
  orginal_image = np.array(orginal_image)
  test_image = cv2.resize(orginal_image,(320,256))
  test_image = np.float32(test_image)/255
  lst = []
  lst.append(test_image)
  img = np.array(lst)
  predicted_mask = model.predict(img)
  predicted_mask = tf.argmax(predicted_mask, axis=-1)
  predicted_mask = tf.squeeze(predicted_mask, axis=0) 
  figure = plot_predicted_mask(orginal_image,predicted_mask)
  plot = Image.open(figure)
  st.image(plot)

Overwriting app.py


In [5]:
!ngrok authtoken 2GlIgpUei0zMptzNQopwoWlM1K0_5RivwJz8MEUpjXD7nGAWK

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

INFO:pyngrok.ngrok:Opening tunnel named: http-80-9f6f17eb-232c-4a10-989a-9c89686a2785
2022-10-31 11:18:35.226 INFO    pyngrok.ngrok: Opening tunnel named: http-80-9f6f17eb-232c-4a10-989a-9c89686a2785
INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:35+0000 lvl=info msg="no configuration paths supplied"
2022-10-31 11:18:35.263 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:35+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:35+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-10-31 11:18:35.266 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:35+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:35+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-10-31 11:18:35.270 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:35+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngro

NgrokTunnel: "http://6a62-34-86-237-200.ngrok.io" -> "http://localhost:80"


INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:40+0000 lvl=info msg="join connections" obj=join id=4b68c778c81e l=127.0.0.1:80 r=103.2.234.62:1214
2022-10-31 11:18:40.960 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:40+0000 lvl=info msg="join connections" obj=join id=4b68c778c81e l=127.0.0.1:80 r=103.2.234.62:1214
INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:41+0000 lvl=info msg="join connections" obj=join id=245f936cff0d l=127.0.0.1:80 r=103.2.234.62:1214
2022-10-31 11:18:41.295 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:41+0000 lvl=info msg="join connections" obj=join id=245f936cff0d l=127.0.0.1:80 r=103.2.234.62:1214
INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:41+0000 lvl=info msg="join connections" obj=join id=84872d962f47 l=127.0.0.1:80 r=103.2.234.62:1213
2022-10-31 11:18:41.303 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:41+0000 lvl=info msg="join connections" obj=join id=84872d962f47 l=127.0.0.1:80 r=103.2.234.62:1213
INFO:pyngrok.process.ngrok:t=2022-10-31T

2022-10-31 11:18:50.547634: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


INFO:pyngrok.process.ngrok:t=2022-10-31T11:18:52+0000 lvl=info msg="join connections" obj=join id=7e5a9596b0d7 l=127.0.0.1:80 r=103.2.234.62:1215
2022-10-31 11:18:52.245 INFO    pyngrok.process.ngrok: t=2022-10-31T11:18:52+0000 lvl=info msg="join connections" obj=join id=7e5a9596b0d7 l=127.0.0.1:80 r=103.2.234.62:1215
